<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/0_SequentialModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 06/24/2021-2
from __future__ import absolute_import
from IPython.display import display, Image
import glob, os, shutil, sys, tarfile
from os.path import *

contentPth = os.getcwd()
tensorflowImagesPth = join(contentPth, 'TensorflowImages')

try:
    from google.colab import drive, files
    if exists('/content/sample_data'):
        shutil.rmtree('/content/sample_data')
    if exists(tensorflowImagesPth):
        shutil.rmtree(tensorflowImagesPth)
    
    drive.mount('/content/gdrive', force_remount=True)
    myDrivePth = '/content/gdrive/My Drive'
    imagesPth = join('/content/gdrive/My Drive/TensorflowResizedImages')
    
    os.chdir(myDrivePth)
    from BashColors import C
    from TarfileFunctions import *
    os.chdir(contentPth)
    
    if not exists(tensorflowImagesPth):
        shutil.copytree(imagesPth, tensorflowImagesPth)
    else: print(f'{C.Green}{tensorflowImagesPth}{C.ColorOff} exists')
        
except ModuleNotFoundError as err: 
    # print(err)
    from BashColors import C
    from TarfileFunctions import *
    if not exists(tensorflowImagesPth):
        tar = tarfile.open(
            'New_Jovyan_Defcon_Augmented_Images.tar.gz')
        tar.extractall()
        tar.close()
    else: print(f'{C.Green}{tensorflowImagesPth}{C.ColorOff} exists')
    
try:
    import tensorflow as tf
    print(f'tensorflow: {C.IBlue}{tf.__version__}{C.ColorOff}')
except ModuleNotFoundError:
    %pip install tensorflow
    import tensorflow as tf
    
try:
    import tensorflow_datasets as tfds
except ModuleNotFoundError:
    %pip install tensorflow-datasets
    import tensorflow_datasets as tfds
    
try:
    import numpy as np
    # np.set_printoptions(precision=4)
except ModuleNotFoundError:
    %pip install numpy
    import numpy as np
    np.set_printoptions(precision=4)

try:
    import PIL
    import PIL.Image
except ModuleNotFoundError:
    %pip install pillow
    import PIL
    import PIL.Image

try:
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    mpl.rcParams['figure.figsize'] = (11, 11)
    mpl.rcParams['axes.grid'] = False
except ModuleNotFoundError:
    %pip install matplotlib
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    mpl.rcParams['figure.figsize'] = (11, 11)
    mpl.rcParams['axes.grid'] = False

try: import pydot
except ModuleNotFoundError as err:
    %pip install pydot
    import pydot
    
try: import graphviz
except Exception as err:
    %pip install graphviz
    import graphviz
    print(f'{C.IRed}{err}{C.ColorOff}')

print(f'cwd: {C.IBlue}{os.getcwd()}')

In [ ]:
# Augment images

In [ ]:
if exists(myDrivePth):
    os.chdir(myDrivePth)
    # %ls *.gz -l
    # tff.listTarfiles()
    # tff.inspectTarfile(named='Defcon_Augmented_Images.tar.gz')
    # tff.extractTarfiles(fileName='Defcon_Augmented_Images.tar.gz')
    os.chdir(contentPth)

In [ ]:
from tensorflow.config.experimental import list_physical_devices
from tensorflow.config.experimental import set_memory_growth
physicalDevices = list_physical_devices()
try:
    print(physicalDevices)
    set_memory_growth(physicalDevices[1], True)
except: pass

In [ ]:
import tensorflow
from tensorflow.keras.utils import plot_model

def plotModel(thisModel):
    fileName = thisModel.name + '.png'
    print(fileName)
    plot_model(model=thisModel,
               to_file=fileName,
               show_shapes=True,
               show_dtype=True,
               expand_nested=True)
    img = Image(filename=fileName)
    display(img)
print('ok')

In [ ]:
from tensorflow.keras.preprocessing import image

def prepare_image(filePath):
    img = image.load_img(path=filePath, target_size=(224, 224))
    image_array = image.img_to_array(img)
    image_array_expanded_dims = np.expand_dims(image_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(
        image_array_expanded_dims
    )
print('ok')

In [ ]:
from IPython.display import Image
def showImage(fromPath):
    showImage = Image(filename=fromPath,
                 width=64, height=64,
                 retina=False)
    display(showImage)
print('ok')

In [ ]:
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution
# tf.config.run_functions_eagerly('True')
print('executing_eagerly:', tf.executing_eagerly())

In [ ]:
batch_size = 8
img_height = 224
img_width = 224

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  tensorflowImagesPth,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  tensorflowImagesPth,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_dataset.class_names
print(class_names[0])
print(class_names[1])

In [ ]:
for images, labels in train_dataset.take(1):
  for i in range(8):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    # title=labels[i].numpy().astype('uint8')
    title=labels[i]
    # plt.title(label=title)
    # plt.title([labels[i]])
    plt.axis("off")


You can train a model using these datasets by passing them to `model.fit` (shown later in this tutorial). If you like, you can also manually iterate over the dataset and retrieve batches of images:

In [ ]:
for image_batch, labels_batch in train_dataset:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

The `image_batch` is a tensor of the shape `(32, 180, 180, 3)`. This is a batch of 32 images of shape `180x180x3` (the last dimension referes to color channels RGB). The `label_batch` is a tensor of the shape `(32,)`, these are corresponding labels to the 32 images. 


Note: you can call `.numpy()` on either of these tensors to convert them to a `numpy.ndarray`.

### Standardize the data


The RGB channel values are in the `[0, 255]` range. This is not ideal for a neural network; in general you should seek to make your input values small. Here, we will standardize values to be in the `[0, 1]` by using a Rescaling layer.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
normalization_layer = Rescaling(scale=1./127.5,offset=-1)

There are two ways to use this layer. You can apply it to the dataset by calling map:

In [ ]:
normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), 'to', np.max(first_image)) 

Or, you can include the layer inside your model definition to simplify deployment. We will use the second approach here.

Note: If you would like to scale pixel values to `[-1,1]` you can instead write `Rescaling(1./127.5, offset=-1)`

Note: we previously resized images using the `image_size` argument of `image_dataset_from_directory`. If you want to include the resizing logic in your model, you can use the [Resizing](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Resizing) layer instead.


### Configure the dataset for performance

Let's make sure to use buffered prefetching so we can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data.

`.cache()` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

`.prefetch()` overlaps data preprocessing and model execution while training. 

Interested readers can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

### Train a model

For completeness, we will show how to train a simple model using the datasets we just prepared. This model has not been tuned in any way - the goal is to show you the mechanics using the datasets you just created. To learn more about image classification, visit this [tutorial](https://www.tensorflow.org/tutorials/images/classification).

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  # layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.Rescaling(1/127.5, offset=-1),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])
print(1./255, 'ok')

In [ ]:
model.build(input_shape=(8, 224, 224, 3))
print('model output shape:' ,model.output_shape)
# model.summary()

In [ ]:
model.compile(
  optimizer = 'adam',
  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics = ['accuracy'])

# model.summary()

In [ ]:
model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=2
)

In [ ]:
from tensorflow.keras.applications import imagenet_utils
os.chdir(tensorflowImagesPth)
imageGlob=glob.glob('**', recursive=True)

for fil in sorted(imageGlob):
    if isfile(fil) and fil.endswith('.png'):
        newFilPath=abspath(fil)
        prepared_image = prepare_image(filePath=newFilPath)
        print(basename(newFilPath))
        showImage(newFilPath)
        # print(prepared_image)
        # thisPrediction = mobile.predict(prepared_image)
        # result = imagenet_utils.decode_predictions(thisPrediction)
        # print(result)
        print()

os.chdir(contentPth)

Note: you can also write a custom training loop instead of using `model.fit`. To learn more, visit this [tutorial](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch).

You may notice the validation accuracy is low to the compared to the training accuracy, indicating our model is overfitting. You can learn more about overfitting and how to reduce it in this [tutorial](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

## Using tf.data for finer control

The above keras.preprocessing utilities are a convenient way to create a `tf.data.Dataset` from a directory of images. For finer grain control, you can write your own input pipeline using `tf.data`. This section shows how to do just that, beginning with the file paths from the zip we downloaded earlier.

In [ ]:
data_dir=tensorflowImagesPth
searchString=join(tensorflowImagesPth, '*/*')
searchString=str(searchString)
datasetList=tf.data.Dataset.list_files(searchString)
print(f'datasetList: {len(datasetList)}')

image_count = len(datasetList)
datasetList = datasetList.shuffle(buffer_size=image_count,
                                  reshuffle_each_iteration=False,)

for fil in datasetList.take(len(datasetList)):
    print(fil.numpy())

In [ ]:
val_size = int(image_count * 0.2)
train_dataset = datasetList.skip(val_size)
validation_dataset = datasetList.take(val_size)

You can see the length of each dataset as follows:

In [ ]:
print(tf.data.experimental.cardinality(train_dataset).numpy())
print(tf.data.experimental.cardinality(validation_dataset).numpy())

In [ ]:
def get_label(file_path): # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.argmax(one_hot)

def decode_img(img): # convert the compressed string to a 3D uint8 tensor
    # img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.decode_png(img, channels=3)
    # resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def getLabelName(forPath):
    img, label = process_path(forPath)
    index = label.numpy()
    return class_names[index]


In [ ]:
os.chdir(tensorflowImagesPth)

for fil in sorted(imageGlob):
    # print(fil)
    fullPath=abspath(fil)
    if fullPath.endswith('.png'):
        # print(fullPath)
        showImage(fullPath)
        className = getLabelName(fullPath)
        print(className)

In [ ]:

enemyPth = join(tensorflowImagesPth, 'Enemy')
enemyPth = join(enemyPth, 'enemy_1.png')
if exists(enemyPth):
    showImage(enemyPth)
    print(getLabelName(enemyPth))



In [ ]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_dataset = train_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in train_dataset.take(9):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy(), '\n')

### Configure dataset for performance

To train a model with this dataset you will want the data:

* To be well shuffled.
* To be batched.
* Batches to be available as soon as possible.

These features can be added using the `tf.data` API. For more details, see the [Input Pipeline Performance](../../guide/performance/datasets) guide.

In [ ]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_dataset = configure_for_performance(train_dataset)
validation_dataset = configure_for_performance(validation_dataset)

In [ ]:
model.fit(train_dataset,
          validation_data=validation_dataset,
          epochs=3)

In [ ]:
modelName = model.name + '_1.00.h5'
print(modelName)
modelSavePath = join(contentPth, modelName)
print(f'{C.IBlue}{modelSavePath}')
model.save(filepath=modelSavePath)
if exists(myDrivePth):
    modelSavePath = join(myDrivePth, modelName)
    print(modelSavePath)
    model.save(modelSavePath)

In [ ]:
from tensorflow.keras.applications import imagenet_utils
os.chdir(tensorflowImagesPth)
imageGlob=glob.glob('**', recursive=True)

trainedModel = tf.keras.models.load_model(filepath=modelSavePath)
# trainedModel.summary()
for fil in sorted(imageGlob):
    if fil.endswith('.png'):
        # print(fil)
        newFilPath= abspath(fil)
        # print(newFilPath)
        # prepared_image = prepare_image(filePath=newFilPath)
        print(basename(newFilPath))
        showImage(newFilPath)

        # print(prepared_image)
        # prediction = trainedModel.predict(newFilPath)
        # result = imagenet_utils.decode_predictions(prediction)
        # result = np.argmax(prediction)
        # print(result)
        print()

os.chdir(contentPth)

In [ ]:
train_dataset = configure_for_performance(train_dataset)
validation_dataset = configure_for_performance(validation_dataset)
# test_ds = configure_for_performance(test_ds)